In [21]:
import os
osp = os.path
from icecream import ic

from pydub import AudioSegment

from mutagen.mp3 import MP3
from mutagen.easyid3 import EasyID3
from mutagen.id3 import ID3

import PySimpleGUI as sg

In [22]:
## VARIABLE SETUP
EXIT_ALL = 'EXIT_ALL'
LAUNCH_SPLITTER = 'LAUNCH_SPLITTER'
LAUNCH_EDITOR = 'LAUNCH_EDITOR'

SPLITTER_LOAD_MP3 = 'SPLITTER_LOAD_MP3'
NUM_TRACKS = 'NUM_TRACKS'
SPLITTER_SETUP_ENTER = 'SPLITTER_SETUP_ENTER'
LOADED_TRACK_TEXT_SETUP = 'LOADED_TRACK_TEXT_SETUP'
LOADED_TRACK_TEXT = 'LOADED_TRACK_TEXT'
SPLITTER_SETUP_CLOSE = 'SPLITTER_SETUP_CLOSE'

START_TIME_OR_DURATION_RADIO_GROUP = 'START_TIME_OR_DURATION_RADIO_GROUP'
START_TIME_RADIO = 'START_TIME_RADIO'
DURATION_RADIO = 'DURATION_TIME_RADIO'
SPLITTER_BACK = 'SPLITTER_BACK'
SPLITTER_CLOSE = 'SPLITTER_CLOSE'
SPLITTER_VALUE = 'SPLITTER_VALUE'
SPLITTER_TABLE = 'SPLITTER_TABLE'
SPLITTER_SUBMIT = 'SPLITTER_SUBMIT'


EDITOR_SETUP_CLOSE = 'EDITOR_SETUP_CLOSE'
EDITOR_SETUP_ENTER = 'EDITOR_SETUP_ENTER'
EDITOR_LOAD_MP3 = 'EDITOR_LOAD_MP3'
EDITOR_SETUP_TABLE = 'EDITOR_SETUP_TABLE'

EDITOR_BACK = 'EDITOR_BACK'
EDITOR_CLOSE = 'EDITOR_CLOSE'
EDITOR_SUBMIT = 'EDITOR_SUBMIT'


splitterFileName = ''
SPLITTERHEADER = ('Title', 'Start Time', 'Duration')

EDITOR_HEADER = ('tracknumber', 'title', 'album', 'albumartist', 'artist', 'date', 'genre')


sg.theme('Topanga');

In [23]:
def make_main_window() -> sg.Window:
    layout = [[sg.Text('Audio Editor')],
              [sg.Button('File Splitter', key=LAUNCH_SPLITTER), sg.Button('Metadata Editor', key=LAUNCH_EDITOR)],
              [sg.Button('Exit', key=EXIT_ALL)]]

    return sg.Window('Main Window', layout, finalize=True)

def make_splitter_setup_window() -> sg.Window:
    layout = [[sg.Text('File Splitter Setup')],
              [sg.Input('', visible=False, enable_events=True, key=SPLITTER_LOAD_MP3), sg.FileBrowse('Load mp3...', initial_folder=osp.abspath('D:\Music')),
               sg.Text('Num Tracks: '), sg.Input('2', size=4, key=NUM_TRACKS)],
               [sg.Text('', key=LOADED_TRACK_TEXT_SETUP)],
               [sg.Button('Close', key=SPLITTER_SETUP_CLOSE), sg.Push(), sg.Button('Enter', key=SPLITTER_SETUP_ENTER, size=(30, 2)), sg.Push()]]
    
    return sg.Window('File Splitter Setup', layout, finalize=True)

def make_splitter_window(loaded_track_name: str, num_tracks: int, header: list[str]) -> sg.Window:
    # A HIGHLY unusual layout definition
    # Normally a layout is specified 1 ROW at a time. Here multiple rows are being contatenated together to produce the layout
    # Note the " + \ " at the ends of the lines rather than the usual " , "
    # This is done because each line is a list of lists
    layout =[[sg.Text('Loaded Track:')],
              [sg.Text(loaded_track_name, key=LOADED_TRACK_TEXT)],
              [sg.Text(''), sg.Radio('Use Start Time', START_TIME_OR_DURATION_RADIO_GROUP, key=START_TIME_RADIO, default=True), sg.Radio('Use Duration', START_TIME_OR_DURATION_RADIO_GROUP, key=DURATION_RADIO)],
              [sg.Column([[sg.Text('', size=4)]] + [[sg.Text(n, size=4)] for n in range(1, num_tracks + 1)]), # Index column
              *[sg.Column([[sg.Push(), sg.Text(col_name), sg.Push()]] + \
                          [[sg.Input('', key=(i_row, i_col), size=20)] for i_row in range(1, num_tracks + 1)]) for i_col, col_name in enumerate(header)],],
              [sg.Text('')],
              [sg.Button('Back', key=SPLITTER_BACK), sg.Text('', size=4), sg.Button('Close', key=SPLITTER_CLOSE), sg.Push(), sg.Button('Submit', key=SPLITTER_SUBMIT, size=(30, 2), font=20)]]
    
    return sg.Window('File Splitter', layout,  default_element_size=(12, 1), element_padding=(1, 1), return_keyboard_events=True, finalize=True)

def make_metadata_editor_setup_window() -> sg.Window:
    layout = [[sg.Text('Metadata Editor Setup')],
              [sg.Table([[]], ['File Name'], key=EDITOR_SETUP_TABLE, col_widths=[30, 30], num_rows=10), # TODO: how to make table wider?
               sg.Input('', visible=False, enable_events=True, key=EDITOR_LOAD_MP3), sg.FilesBrowse('Select mp3(s)...', initial_folder=osp.abspath('D:\Music'))],
               [sg.Text('', key=LOADED_TRACK_TEXT_SETUP)],
               [sg.Button('Close', key=EDITOR_SETUP_CLOSE), sg.Push(), sg.Button('Enter', key=EDITOR_SETUP_ENTER, size=(30, 2)), sg.Push()]]
    
    return sg.Window('Metadata Editor Setup', layout, finalize=True, resizable=True)

def make_metadata_editor_window(num_tracks: int, header: list[str]) -> sg.Window:    
    layout =[[sg.Text('')],
            [sg.Push(), sg.Text('Apply to all with ENTER on top row'), sg.Push()],
            [sg.Column([[sg.Text('0', size=4)]] + [[sg.Text('', size=4)]] + [[sg.Text(n, size=4)] for n in range(1, num_tracks + 1)]), # Index column
            *[sg.Column([[sg.Input('', key=(0, i_col), size=20)]] + \
                        [[sg.Push(), sg.Text(col_name), sg.Push()]] + \
                        [[sg.Input('', key=(i_row, i_col), size=20)] for i_row in range(1, num_tracks + 1)]) for i_col, col_name in enumerate(header)],],
            [sg.Text('')],
            [sg.Button('Back', key=EDITOR_BACK), sg.Text('', size=4), sg.Button('Close', key=EDITOR_CLOSE), sg.Push(), sg.Button('Submit', key=EDITOR_SUBMIT, size=(30, 2), font=20)]]
    
    return sg.Window('Metadata Editor', layout,  default_element_size=(12, 1), element_padding=(1, 1), return_keyboard_events=True, finalize=True)


def get_msec(time_str: str) -> int:
    """Get seconds from time
        time can be specified with colon or comma
    """
    if '.' in time_str:
        split_char = '.'
    elif ':' in time_str:
        split_char = ':'
    else:
        split_char = None
        
    secs = 1000 * sum(int(x) * 60 ** i for i, x in enumerate(reversed(time_str.split(split_char))))
    
    return secs
    

In [24]:
mainWindow, splitterSetupWindow, splitterWindow, editorSetupWindow, editorWindow = make_main_window(), None, None, None, None

while True:
    window, event, values = sg.read_all_windows()
    
    if window == mainWindow and event in (sg.WIN_CLOSED, EXIT_ALL):
        break

    if window == mainWindow:
        if event == LAUNCH_SPLITTER:
            splitterSetupWindow = make_splitter_setup_window()
            mainWindow[LAUNCH_SPLITTER].update(disabled=True)
        if event == LAUNCH_EDITOR:
            editorSetupWindow = make_metadata_editor_setup_window()
            mainWindow[LAUNCH_EDITOR].update(disabled=True)

    if window == splitterSetupWindow:
        if event in (sg.WIN_CLOSED, SPLITTER_SETUP_CLOSE):
            splitterSetupWindow.close()
            mainWindow[LAUNCH_SPLITTER].update(disabled=False)
        elif event == SPLITTER_LOAD_MP3:
            splitterFilePath = osp.abspath(values[SPLITTER_LOAD_MP3])
            splitterRawAudio : AudioSegment = AudioSegment.from_mp3(splitterFilePath)
            splitterFileName = osp.basename(splitterFilePath)
            splitterSetupWindow[LOADED_TRACK_TEXT_SETUP].update(f"Loaded: {splitterFileName}")
        elif event == SPLITTER_SETUP_ENTER:
            if splitterFileName == '':
                sg.popup('Load file first')
            else:
                numSplitterTracks = int(values[NUM_TRACKS])
                splitterWindow = make_splitter_window(splitterFileName, numSplitterTracks, SPLITTERHEADER)
                splitterSetupWindow.hide()

    if window == splitterWindow:
        elem : sg.Element = window.find_element_with_focus()
        current_cell : tuple[int, int] = elem.Key if elem and type(elem.Key) is tuple else (0, 0)
        row, col = current_cell

        if event in (sg.WIN_CLOSED, SPLITTER_BACK):
            splitterWindow.close()
            splitterSetupWindow.un_hide()
        elif event == SPLITTER_CLOSE:
            splitterWindow.close()
            splitterSetupWindow.write_event_value(SPLITTER_SETUP_CLOSE, None)
        elif event == SPLITTER_SUBMIT:
            # check title is input for all tracks
            # split and export songs
            startTime = 0
            for trackNum in range(1, 1 + numSplitterTracks): # trackNum is row index
                trackName = values[(trackNum, 0)] # TODO: hard-coded column 1 for 'Title' column
                
                if values[DURATION_RADIO]:
                    if trackNum != numSplitterTracks:
                        stopTime = startTime + get_msec(values[(trackNum, 2)]) # TODO: hard-coded column 2 for 'Duration' column
                    else: # last row
                        stopTime = len(splitterRawAudio) # get end of file since no next start time
                else: # START_TIME_RADIO
                    startTime = get_msec(values[(trackNum, 1)]) # TODO: hard-coded column 1 for 'Start Time' column
                    if trackNum != numSplitterTracks:
                        stopTime = get_msec(values[(1 + trackNum, 1)]) # get next start time # TODO: hard-coded column 1 for 'Start Time' column
                    else: # last row
                        stopTime = len(splitterRawAudio) # get end of file since no next start time
                
                songAudio : AudioSegment = splitterRawAudio[startTime:stopTime]
                songAudio.export(osp.join(osp.dirname(splitterFilePath) , f"{trackName}.mp3"), format='mp3')
                startTime = stopTime # set start time to stop time for duration input method. will be overwritten if using start time input method
        # events for arrow key inputs
        elif event.startswith('Down'):
            row = row + 1 * (row < numSplitterTracks)
        elif event.startswith('Left'):
            col = col - 1 * (col > 0)
        elif event.startswith('Right'):
            col = col + 1 * (col < len(SPLITTERHEADER) - 1)
        elif event.startswith('Up'):
            row = row - 1 * (row > 0)
        # if the current cell changed, set focus on new cell
        if current_cell != (row, col):
            current_cell = row, col
            window[current_cell].set_focus()          # set the focus on the element moved to
            window[current_cell].update(select=True)  # when setting focus, also highlight the data in the element so typing overwrites
        # enter keypress
        if event == '\r':
            if row == 0:
                # set active column input boxes to row 0 input box value
                for r in range(1, 1 + numSplitterTracks):
                    window[(r, col)].update(values[(0, col)])
        

    if window == editorSetupWindow:
        if event in (sg.WIN_CLOSED, EDITOR_SETUP_CLOSE):
            editorSetupWindow.close()
            mainWindow[LAUNCH_EDITOR].update(disabled=False)
        elif event == EDITOR_LOAD_MP3:
            editorFilePaths = [osp.abspath(pathStr) for pathStr in values[EDITOR_LOAD_MP3].split(';')]
            editorFileNames = [osp.basename(path) for path in editorFilePaths]
            editorSetupWindow[EDITOR_SETUP_TABLE].update(values=[[name] for name in editorFileNames])
        elif event == EDITOR_SETUP_ENTER:
            if values[EDITOR_LOAD_MP3] == '': # check if nothing selected
                sg.popup('Load file(s) first')
            numEditorTracks = len(editorFileNames)
            editorWindow = make_metadata_editor_window(num_tracks=numEditorTracks, header=EDITOR_HEADER)
            # generate mp3 files
            mp3Files = [MP3(songPath, ID3=EasyID3) for songPath in editorFilePaths]
            # update inputs with loaded song(s)' metadata
            for j, (file, name) in enumerate(zip(mp3Files, editorFileNames)):
                for i, header in enumerate(EDITOR_HEADER):
                    try:
                        editorWindow[(j + 1, i)].update(file[header][0])
                    except KeyError: # don't update if ID3 field isn't present (except for title. update that to be filename)
                        if header == 'title':
                            editorWindow[(j + 1, i)].update(name[:-4]) # TODO: hard-coded -4 to remove .mp3
            editorSetupWindow.hide()
            

    if window == editorWindow:
        elem : sg.Element = window.find_element_with_focus()
        current_cell : tuple[int, int] = elem.Key if elem and type(elem.Key) is tuple else (0, 0)
        row, col = current_cell

        if event in (sg.WIN_CLOSED, EDITOR_BACK):
            editorWindow.close()
            editorSetupWindow.un_hide()
            mainWindow[LAUNCH_EDITOR].update(disabled=False)
        elif event == EDITOR_CLOSE:
            editorWindow.close()
            editorSetupWindow.write_event_value(EDITOR_SETUP_CLOSE, None)
        elif event == EDITOR_SUBMIT:
            # update metadata and save mp3 files
            for j, file in enumerate(mp3Files):
                for i, header in enumerate(EDITOR_HEADER):
                    file[header] = [values[(j + 1, i)]]
                file.save()
                filePath = file.__dict__['filename']
                os.rename(filePath, osp.join(osp.abspath(filePath), osp.join(osp.dirname(filePath), f"{file['title'][0]}.mp3")))# change the file name to match title
            editorWindow.write_event_value(EDITOR_BACK, None)

        # events for arrow key inputs
        elif event.startswith('Down'):
            row = row + 1 * (row < numEditorTracks)
        elif event.startswith('Left'):
            col = col - 1 * (col > 0)
        elif event.startswith('Right'):
            col = col + 1 * (col < len(SPLITTERHEADER) - 1)
        elif event.startswith('Up'):
            row = row - 1 * (row > 0)
        # if the current cell changed, set focus on new cell
        if current_cell != (row, col):
            current_cell = row, col
            window[current_cell].set_focus()          # set the focus on the element moved to
            window[current_cell].update(select=True)  # when setting focus, also highlight the data in the element so typing overwrites
        # enter keypress
        if event == '\r':
            if row == 0:
                # set active column input boxes to row 0 input box value
                for r in range(1, 1 + numEditorTracks):
                    window[(r, col)].update(values[(0, col)])


# close all windows
windows = (mainWindow, splitterSetupWindow, splitterWindow, editorSetupWindow, editorWindow)
for win in windows:
    if win is not None:
        win.close()

AttributeError: 'NoneType' object has no attribute 'write_event_value'